In [1]:
# import os

# os.chdir("/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/")

In [3]:
print("hhhh")

hhhh


In [5]:
# import findspark
# findspark.init()

In [4]:
from pyspark import SparkContext 
sc = SparkContext.getOrCreate() 

In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("KafkaConnectionTest") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
    .getOrCreate()

df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "35.206.252.44:9092") \
    .option("subscribe", "test-topic") \
    .load()

df.printSchema()


AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.

In [8]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pyspark.sql.functions import col, from_json, to_date, avg, lit, to_timestamp
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import time
import shutil
from datetime import datetime, timedelta
import os
from pyspark.sql import functions as F

gcs_jar_path = os.path.abspath(r"D:\Empty\crypto-big-data\config\gcs-connector-hadoop3-latest.jar")

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("hehee") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")\
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")\
    .config("spark.hadoop.fs.gs.auth.service.account.enable", "true")\
    .config("spark.hadoop.fs.gs.auth.service.account.json.keyfile", r"D:\Empty\crypto-big-data\config\btcanalysishust-d7c3a4830bef.json") \
    .config("spark.jars", gcs_jar_path) \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0,io.delta:delta-core_2.12:2.4.0")\
    .config("spark.hadoop.fs.gs.project.id", "btcanalysishust")\
    .getOrCreate()


In [9]:
from pyspark.sql.types import StructType, StructField, TimestampType, DoubleType


In [10]:
# Schema của dữ liệu Kafka
schema = StructType([
    StructField("timestamp", StringType()),
    StructField("prices", StructType([
        StructField("bitcoin", FloatType()),
        StructField("ethereum", FloatType()),
        StructField("tether", FloatType()),
        StructField("usd-coin", FloatType()),
        StructField("ripple", FloatType()),
        StructField("cardano", FloatType()),
        StructField("dogecoin", FloatType()),
        StructField("matic-network", FloatType()),
        StructField("solana", FloatType()),
        StructField("litecoin", FloatType()),
        StructField("polkadot", FloatType()),
        StructField("shiba-inu", FloatType()),
        StructField("tron", FloatType()),
        StructField("cosmos", FloatType()),
        StructField("chainlink", FloatType()),
        StructField("stellar", FloatType()),
        StructField("near", FloatType()),
    ]))
])

# Đọc dữ liệu từ Kafka
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "35.206.252.44:9092") \
    .option("subscribe", "crypto-pricessss") \
    .option("startingOffsets", "latest") \
    .option("maxOffsetsPerTrigger", 1000)\
    .load()


parsed_df = kafka_df.selectExpr("CAST(value AS STRING)") \
    .select(F.from_json(F.col("value"), schema).alias("data"))

crypto_parsed_df = parsed_df.select(
    F.to_timestamp(F.col("data.timestamp"), "yyyy-MM-dd'T'HH:mm:ss.SSSSSSXXX").alias("DATE"),
    F.col("data.prices.bitcoin").alias("CLOSE")
)

    # F.col("data.prices.ethereum").alias("ETH"),
    # F.col("data.prices.tether").alias("USDT"),
    # F.col("data.prices.usd-coin").alias("USDC"),
    # F.col("data.prices.ripple").alias("XRP"),
    # F.col("data.prices.cardano").alias("ADA"),
    # F.col("data.prices.dogecoin").alias("DOGE"),
    # F.col("data.prices.matic-network").alias("MATIC"),
    # F.col("data.prices.solana").alias("SOL"),
    # F.col("data.prices.litecoin").alias("LTC"),
    # F.col("data.prices.polkadot").alias("DOT"),
    # F.col("data.prices.shiba-inu").alias("SHIB"),
    # F.col("data.prices.tron").alias("TRX"),
    # F.col("data.prices.cosmos").alias("ATOM"),
    # F.col("data.prices.chainlink").alias("LINK"),
    # F.col("data.prices.stellar").alias("XLM"),
    # F.col("data.prices.near").alias("NEAR")
# )

# # Định nghĩa schema cho dữ liệu lịch sử
historical_schema = StructType([
    StructField("BASE", StringType(), True),#là mã của crypto, ví dụ BASE BTC sẽ có các row chứa dữ liệu theo các cột kế bên
    StructField("DATE", TimestampType(), True),
    StructField("OPEN", FloatType(), True),
    StructField("HIGH", FloatType(), True),
    StructField("LOW", FloatType(), True),
    StructField("CLOSE", FloatType(), True),
    StructField("VOLUME", FloatType(), True),
    StructField("YEAR", StringType(), True),
    StructField("MONTH", StringType(), True),
    StructField("__index_level_0__", StringType(), True)
])


AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.

In [5]:
# 1. Đọc dữ liệu batch từ Parquet
historical_data_df = (
    spark.read.format("parquet")
    .load("gs://crypto-historical-data-2/ver2/BTC/2024/*")
    .select(F.col("DATE").cast("timestamp"), "CLOSE")
)

# 3. Trigger micro-batch và xử lý
def process_batch(micro_batch_df, batch_id):
    # Lọc ra giá trị gần nhất của micro-batch (dòng có timestamp mới nhất)
    micro_batch_latest_df = (
        micro_batch_df
        .withColumn("row_num", F.row_number().over(Window.orderBy(F.col("DATE").desc())))
        .filter(F.col("row_num") == 1)  # Chỉ lấy dòng có timestamp mới nhất
        .drop("row_num")  # Loại bỏ cột row_num
    )
    # Kết hợp micro-batch với dữ liệu batch (historical_data_df)
    combined_df = (
        micro_batch_latest_df
        .unionByName(historical_data_df)  # Union với dữ liệu batch
    )
    
    # Tính toán các chỉ số SMA với các cửa sổ khác nhau
    window_spec = Window.orderBy("DATE").rowsBetween(Window.unboundedPreceding, 0)

    combined_df = combined_df.withColumn("SMA_5", F.avg("CLOSE").over(window_spec.rowsBetween(-4, 0)))
    combined_df = combined_df.withColumn("SMA_10", F.avg("CLOSE").over(window_spec.rowsBetween(-9, 0)))
    combined_df = combined_df.withColumn("SMA_20", F.avg("CLOSE").over(window_spec.rowsBetween(-19, 0)))
    combined_df = combined_df.withColumn("SMA_50", F.avg("CLOSE").over(window_spec.rowsBetween(-49, 0)))
    combined_df = combined_df.withColumn("SMA_100", F.avg("CLOSE").over(window_spec.rowsBetween(-99, 0)))
    combined_df = combined_df.withColumn("SMA_200", F.avg("CLOSE").over(window_spec.rowsBetween(-199, 0)))

    # Sắp xếp lại theo DATE
    combined_df = combined_df.orderBy("DATE", ascending=False)

    # Ghi ra kết quả
    combined_df.write.format("console").option("truncate", False).save()
    # combined_df.write \
    # .format("csv") \
    # .option("header", "true") \
    # .option("path", "gs://indicator-crypto/sma_results/BTC") \
    # .mode("overwrite") \
    # .save()


query = crypto_parsed_df.writeStream \
    .foreachBatch(process_batch) \
    .start()

query.awaitTermination()


24/12/13 22:40:44 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/k9/xss4jl9s24sg7_7zhcmrp2g00000gn/T/temporary-c0ebada1-cde1-4576-8139-dd3b88b955ad. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/12/13 22:40:44 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/12/13 22:40:45 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
24/12/13 22:40:46 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eve

+-------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|DATE               |CLOSE   |SMA_5            |SMA_10           |SMA_20           |SMA_50           |SMA_100          |SMA_200          |
+-------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|2024-12-13 00:00:00|100088.2|99074.06         |99215.75         |97511.875        |86722.98399999997|74511.1914       |68735.22729999998|
|2024-12-12 00:00:00|100050.1|99290.42         |98798.68000000001|97458.66650000002|86053.24219999996|74084.9978       |68577.15489999998|
|2024-12-11 00:00:00|101177.0|99260.38         |98379.0          |97378.11600000001|85399.84179999998|73675.8951       |68423.35274999999|
|2024-12-10 00:00:00|96655.1 |98999.55999999998|97907.84199999999|97038.9235       |84723.22479999997|73237.1151       |68260.20674999998|
|2024-12-09 00:00:00|97399.

24/12/13 22:41:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/13 22:41:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|DATE                      |CLOSE   |SMA_5            |SMA_10           |SMA_20           |SMA_50           |SMA_100          |SMA_200          |
+--------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|2024-12-13 22:41:04.301352|101312.0|99856.48000000001|99471.23999999999|97688.5365       |87386.08719999997|74944.6014       |68894.94369999999|
|2024-12-13 00:00:00       |100088.2|99074.06         |99215.75         |97511.875        |86722.98399999997|74511.1914       |68735.22729999998|
|2024-12-12 00:00:00       |100050.1|99290.42         |98798.68000000001|97458.66650000002|86053.24219999996|74084.9978       |68577.15489999998|
|2024-12-11 00:00:00       |101177.0|99260.38         |98379.0          |97378.11600000001|85399.84179999998|73675.8951     

24/12/13 22:42:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/13 22:42:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


...

24/12/13 22:48:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/13 22:48:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|DATE                     |CLOSE   |SMA_5            |SMA_10           |SMA_20           |SMA_50           |SMA_100          |SMA_200          |
+-------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|2024-12-13 22:48:04.29577|101180.0|99830.08         |99458.04         |97681.9365       |87383.44719999997|74943.28139999999|68894.28369999999|
|2024-12-13 00:00:00      |100088.2|99074.06         |99215.75         |97511.875        |86722.98399999997|74511.1914       |68735.22729999998|
|2024-12-12 00:00:00      |100050.1|99290.42         |98798.68000000001|97458.66650000002|86053.24219999996|74084.9978       |68577.15489999998|
|2024-12-11 00:00:00      |101177.0|99260.38         |98379.0          |97378.11600000001|85399.84179999998|73675.8951       |6842

24/12/13 22:49:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/13 22:49:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/spark-env/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ibulmnie/Documents/20241/BigData/crypto-big-data/spark-env/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/sock

+--------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|DATE                      |CLOSE   |SMA_5            |SMA_10           |SMA_20           |SMA_50           |SMA_100          |SMA_200          |
+--------------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|2024-12-13 22:49:05.188617|100631.0|99720.28         |99403.13999999998|97654.4865       |87372.46719999997|74937.7914       |68891.53869999998|
|2024-12-13 00:00:00       |100088.2|99074.06         |99215.75         |97511.875        |86722.98399999997|74511.1914       |68735.22729999998|
|2024-12-12 00:00:00       |100050.1|99290.42         |98798.68000000001|97458.66650000002|86053.24219999996|74084.9978       |68577.15489999998|
|2024-12-11 00:00:00       |101177.0|99260.38         |98379.0          |97378.11600000001|85399.84179999998|73675.8951     

KeyboardInterrupt: 

In [26]:
from google.cloud import storage
from google.oauth2 import service_account

KEY_FILE = r"D:\Empty\btcanalysishust-d7c3a4830bef.json"


credentials = service_account.Credentials.from_service_account_file(KEY_FILE)

client = storage.Client(
    credentials=credentials, 
    project='btcanalysishust'
)


bucket = client.get_bucket("crypto-historical-data-2")
blob = bucket.blob("ver2/ADA/2021/03/data.parquet")
print(blob.exists())  # Should return True if the file exists
# bucket

True


In [27]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DoubleType, IntegerType, LongType
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

import os
import sys

# Explicitly set Python and library paths
python_path = sys.executable
spark_home = r"D:\Empty\spark-3.5.3-bin-hadoop3-scala2.13\spark-3.5.3-bin-hadoop3-scala2.13"  # Adjust to your Spark installation path
hadoop_home = r"D:\Empty\hadoop\hadoop"
java_home = r"D:\Empty\Java"
spark_temp = r"D:\Empty\spark-temp"

KEY_FILE = r"D:\Empty\btcanalysishust-d7c3a4830bef.json"
# bucket_name = 'ba_bucket3'
gcs_connector_jar = os.path.join(hadoop_home, 'share', 'hadoop', 'tools', 'lib', 'gcs-connector-hadoop3-latest.jar')

# Set environment variables
os.environ['SPARK_HOME'] = spark_home
os.environ['HADOOP_HOME'] = hadoop_home
os.environ['JAVA_HOME'] = java_home
os.environ['PYSPARK_PYTHON'] = python_path
os.environ['PYSPARK_DRIVER_PYTHON'] = python_path

# Add Spark and Python paths
sys.path.append(os.path.join(spark_home, 'python'))
sys.path.append(os.path.join(spark_home, 'python', 'lib', 'py4j-0.10.9.7-src.zip'))

# gcs_jar_path = os.path.abspath("config/gcs-connector-hadoop3-latest.jar")
from pyspark.sql import SparkSession
import sys
# python_path = sys.executable
# gcs_connector_jar = r"D:\Empty\crypto-big-data\config\gcs-connector-hadoop3-latest.jar
# hadoop_home = r"D:\Empty\hadoop\hadoop"
# gcs_connector_jar = os.path.join(hadoop_home, 'share', 'hadoop', 'tools', 'lib', 'gcs-connector-hadoop3-latest.jar')
# KEY_FILE = r"D:\Empty\crypto-big-data\config\btcanalysishust-d7c3a4830bef.json"
# spark_temp = r"D:\Empty\spark-temp"

# Khởi tạo SparkSession
# spark = SparkSession.builder \
#     .appName("hehee") \
#     .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")\
#     .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")\
#     .config("spark.hadoop.fs.gs.auth.service.account.enable", "true")\
#     .config("spark.hadoop.fs.gs.auth.service.account.json.keyfile", KEY_FILE) \
#     .config("spark.driver.extraClassPath", gcs_connector_jar) \
#     .config("spark.executor.extraClassPath", gcs_connector_jar) \
#     .config("spark.jars.packages", 
#             "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,io.delta:delta-core_2.12:2.2.0") \
#     .config("spark.hadoop.fs.gs.project.id", "btcanalysishust")\
#     .getOrCreate()
spark = SparkSession.builder \
    .appName("GCS Connector Test") \
    .config("spark.pyspark.python", python_path) \
    .config("spark.pyspark.driver.python", python_path) \
    .config("spark.driver.extraLibraryPath", os.path.dirname(python_path)) \
    .config("spark.driver.extraClassPath", gcs_connector_jar) \
    .config("spark.executor.extraClassPath", gcs_connector_jar) \
    .config("spark.local.dir", spark_temp) \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", KEY_FILE) \
    .master("local[*]") \
    .getOrCreate()

column_names = ["BTC", "ETH", "USDT", "USDC", "XRP", "ADA", "DOGE", "MATIC", "SOL"]

schema = StructType([
    StructField("BASE", StringType(), True),  
    StructField("DATE", StringType(), True),
    StructField("OPEN", DoubleType(), True),
    StructField("HIGH", DoubleType(), True),
    StructField("LOW", DoubleType(), True),
    StructField("CLOSE", DoubleType(), True),
    StructField("VOLUME", DoubleType(), True),
    StructField("YEAR", IntegerType(), True),
    StructField("MONTH", IntegerType(), True),
    StructField("__index_level_0__", LongType(), True)
])

import datetime

def read_historical_data(coin):
    current_year = datetime.datetime.now().year  
    years = [str(year) for year in range(2021, current_year + 1)]  

    paths = [f"gs://crypto-historical-data-2/ver2/{coin}/{year}/*" for year in years]
    
    return (
        spark.read.schema(schema).parquet(*paths)  
        .select(F.col("DATE").cast("timestamp"), "CLOSE")
    )

def process_coin(coin):
    historical_data_df = read_historical_data(coin)
    window_spec = Window.orderBy("DATE").rowsBetween(Window.unboundedPreceding, 0)
    
    historical_data_df = historical_data_df.withColumn(f"SMA_5", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-4, 0)))
    historical_data_df = historical_data_df.withColumn(f"SMA_10", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-9, 0)))
    historical_data_df = historical_data_df.withColumn(f"SMA_20", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-19, 0)))
    historical_data_df = historical_data_df.withColumn(f"SMA_50", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-49, 0)))
    historical_data_df = historical_data_df.withColumn(f"SMA_100", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-99, 0)))
    historical_data_df = historical_data_df.withColumn(f"SMA_200", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-199, 0)))

    historical_data_df = historical_data_df.select("DATE", "CLOSE", "SMA_5", "SMA_10", "SMA_20", "SMA_50", "SMA_100", "SMA_200").orderBy("DATE", ascending=False)
    return historical_data_df


historical_data_df = process_coin("BTC")
historical_data_df.show()
    # tmp_dir = f"gs://indicator-crypto/sma_results/batch/{coin}"

    # historical_data_df.write \
    #     .format("csv") \
    #     .option("header", "true") \
    #     .option("path", tmp_dir) \
    #     .mode("append") \
    #     .save()

# with ThreadPoolExecutor() as executor:
#     futures = [executor.submit(process_coin, coin) for coin in column_names]
#     for future in as_completed(futures):
#         print(future.result())


Py4JJavaError: An error occurred while calling o577.parquet.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "gs"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:246)
	at scala.collection.immutable.List.map(List.scala:79)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(DataSource.scala:722)
	at org.apache.spa

In [32]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DoubleType, IntegerType, LongType
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

import os
import sys

# Explicitly set Python and library paths
python_path = sys.executable
spark_home = r"D:\Empty\spark-3.5.3-bin-hadoop3-scala2.13\spark-3.5.3-bin-hadoop3-scala2.13"  # Adjust to your Spark installation path
hadoop_home = r"D:\Empty\hadoop\hadoop"
java_home = r"D:\Empty\Java"
spark_temp = r"D:\Empty\spark-temp"

KEY_FILE = r"D:\Empty\btcanalysishust-d7c3a4830bef.json"
# bucket_name = 'ba_bucket3'
gcs_connector_jar = os.path.join(hadoop_home, 'share', 'hadoop', 'tools', 'lib', 'gcs-connector-hadoop3-latest.jar')

# Set environment variables
os.environ['SPARK_HOME'] = spark_home
os.environ['HADOOP_HOME'] = hadoop_home
os.environ['JAVA_HOME'] = java_home
os.environ['PYSPARK_PYTHON'] = python_path
os.environ['PYSPARK_DRIVER_PYTHON'] = python_path

# Add Spark and Python paths
sys.path.append(os.path.join(spark_home, 'python'))
sys.path.append(os.path.join(spark_home, 'python', 'lib', 'py4j-0.10.9.7-src.zip'))

# gcs_jar_path = os.path.abspath("config/gcs-connector-hadoop3-latest.jar")
from pyspark.sql import SparkSession
import sys
# python_path = sys.executable
# gcs_connector_jar = r"D:\Empty\crypto-big-data\config\gcs-connector-hadoop3-latest.jar
# hadoop_home = r"D:\Empty\hadoop\hadoop"
# gcs_connector_jar = os.path.join(hadoop_home, 'share', 'hadoop', 'tools', 'lib', 'gcs-connector-hadoop3-latest.jar')
# KEY_FILE = r"D:\Empty\crypto-big-data\config\btcanalysishust-d7c3a4830bef.json"
# spark_temp = r"D:\Empty\spark-temp"

# Khởi tạo SparkSession
# spark = SparkSession.builder \
#     .appName("hehee") \
#     .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")\
#     .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")\
#     .config("spark.hadoop.fs.gs.auth.service.account.enable", "true")\
#     .config("spark.hadoop.fs.gs.auth.service.account.json.keyfile", KEY_FILE) \
#     .config("spark.driver.extraClassPath", gcs_connector_jar) \
#     .config("spark.executor.extraClassPath", gcs_connector_jar) \
#     .config("spark.jars.packages", 
#             "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,io.delta:delta-core_2.12:2.2.0") \
#     .config("spark.hadoop.fs.gs.project.id", "btcanalysishust")\
#     .getOrCreate()
# spark = SparkSession.builder \
#     .appName("GCS Connector Test") \
#     .config("spark.pyspark.python", python_path) \
#     .config("spark.pyspark.driver.python", python_path) \
#     .config("spark.driver.extraLibraryPath", os.path.dirname(python_path)) \
#     .config("spark.driver.extraClassPath", gcs_connector_jar) \
#     .config("spark.executor.extraClassPath", gcs_connector_jar) \
#     .config("spark.local.dir", spark_temp) \
#     .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
#     .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
#     .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", KEY_FILE) \
#     .master("local[*]") \
#     .getOrCreate()
spark = SparkSession.builder \
    .appName("GCS Connector Test") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
    .config("spark.hadoop.fs.gs.auth.service.account.enable", "true") \
    .config("spark.hadoop.fs.gs.auth.service.account.json.keyfile", KEY_FILE) \
    .config("spark.driver.extraClassPath", gcs_connector_jar) \
    .config("spark.executor.extraClassPath", gcs_connector_jar) \
    .config("spark.hadoop.fs.gs.project.id", "btcanalysishust") \
    .master("local[*]") \
    .getOrCreate()

column_names = ["BTC", "ETH", "USDT", "USDC", "XRP", "ADA", "DOGE", "MATIC", "SOL"]

schema = StructType([
    StructField("BASE", StringType(), True),  
    StructField("DATE", StringType(), True),
    StructField("OPEN", DoubleType(), True),
    StructField("HIGH", DoubleType(), True),
    StructField("LOW", DoubleType(), True),
    StructField("CLOSE", DoubleType(), True),
    StructField("VOLUME", DoubleType(), True),
    StructField("YEAR", IntegerType(), True),
    StructField("MONTH", IntegerType(), True),
    StructField("__index_level_0__", LongType(), True)
])

path = "gs://crypto-historical-data-2/ver2/ADA/2021/03/data.parquet"
df = spark.read.parquet(path)
# blob = bucket.blob("ver2/ADA/2021/03/data.parquet")
df.show()

Py4JJavaError: An error occurred while calling o619.parquet.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "gs"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:246)
	at scala.collection.immutable.List.map(List.scala:79)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(DataSource.scala:722)
	at org.apache.spa

In [37]:
import os
import sys
from pyspark.sql import SparkSession

# Paths (ensure these match your system)
PYTHON_PATH = sys.executable
SPARK_HOME = r"D:\Empty\spark-3.5.3-bin-hadoop3-scala2.13\spark-3.5.3-bin-hadoop3-scala2.13"
HADOOP_HOME = r"D:\Empty\hadoop\hadoop"
JAVA_HOME = r"D:\Empty\Java"
KEY_FILE = r"D:\Empty\btcanalysishust-d7c3a4830bef.json"
GCS_CONNECTOR_JAR = r"D:\Empty\hadoop\hadoop\share\hadoop\tools\lib\gcs-connector-hadoop3-latest.jar"

# Set environment variables
os.environ['SPARK_HOME'] = SPARK_HOME
os.environ['HADOOP_HOME'] = HADOOP_HOME
os.environ['JAVA_HOME'] = JAVA_HOME
os.environ['PYSPARK_PYTHON'] = PYTHON_PATH
os.environ['PYSPARK_DRIVER_PYTHON'] = PYTHON_PATH

# Create Spark session with extensive GCS configuration
spark = SparkSession.builder \
    .appName("GCS_Connector_Debugging") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
    .config("spark.hadoop.fs.gs.auth.service.account.enable", "true") \
    .config("spark.hadoop.fs.gs.auth.service.account.json.keyfile", KEY_FILE) \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", KEY_FILE) \
    .config("spark.hadoop.fs.gs.project.id", "btcanalysishust") \
    .config("spark.driver.extraClassPath", GCS_CONNECTOR_JAR) \
    .config("spark.executor.extraClassPath", GCS_CONNECTOR_JAR) \
    .config("spark.jars", GCS_CONNECTOR_JAR) \
    .master("local[*]") \
    .getOrCreate()

# Path to your Parquet file
path = "gs://crypto-historical-data-2/ver2/ADA/2021/03/data.parquet"

# Import necessary Java classes directly
from pyspark import SparkContext

# Attempt to read the file
try:
    # Create Hadoop Configuration
    hadoop_conf = spark.sparkContext._jsc.hadoopConfiguration()
    hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
    hadoop_conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
    hadoop_conf.set("google.cloud.auth.service.account.enable", "true")
    hadoop_conf.set("google.cloud.auth.service.account.json.keyfile", KEY_FILE)

    # Try reading the file
    df = spark.read.parquet(path)
    print("DataFrame schema:")
    df.printSchema()
    df.show(5)

except Exception as e:
    print(f"Error occurred: {e}")
    import traceback
    traceback.print_exc()

Error occurred: An error occurred while calling o640.parquet.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:246)
	at scala.collection.immutable.List.map(List.scala:79)
	at org.apache

Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_30936\4288638905.py", line 51, in <module>
    df = spark.read.parquet(path)
  File "D:\Empty\spark-3.5.3-bin-hadoop3-scala2.13\spark-3.5.3-bin-hadoop3-scala2.13\python\pyspark\sql\readwriter.py", line 544, in parquet
    return self._df(self._jreader.parquet(_to_seq(self._spark._sc, paths)))
  File "D:\Empty\spark-3.5.3-bin-hadoop3-scala2.13\spark-3.5.3-bin-hadoop3-scala2.13\python\lib\py4j-0.10.9.7-src.zip\py4j\java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "D:\Empty\spark-3.5.3-bin-hadoop3-scala2.13\spark-3.5.3-bin-hadoop3-scala2.13\python\pyspark\errors\exceptions\captured.py", line 179, in deco
    return f(*a, **kw)
  File "D:\Empty\spark-3.5.3-bin-hadoop3-scala2.13\spark-3.5.3-bin-hadoop3-scala2.13\python\lib\py4j-0.10.9.7-src.zip\py4j\protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: An error occurre

In [2]:
import os
import sys

# Explicitly set Python and library paths
python_path = sys.executable
spark_home = r"D:\Empty\spark-3.5.3-bin-hadoop3-scala2.13\spark-3.5.3-bin-hadoop3-scala2.13"  # Adjust to your Spark installation path
hadoop_home = r"D:\Empty\hadoop\hadoop"
java_home = r"D:\Empty\Java"

KEY_FILE = r"D:\Empty\btcanalysishust-d7c3a4830bef.json"
bucket_name = 'ba_bucket3'
gcs_connector_jar = os.path.join(hadoop_home, 'share', 'hadoop', 'tools', 'lib', 'gcs-connector-hadoop3-latest.jar')

# Set environment variables
os.environ['SPARK_HOME'] = spark_home
os.environ['HADOOP_HOME'] = hadoop_home
os.environ['JAVA_HOME'] = java_home
os.environ['PYSPARK_PYTHON'] = python_path
os.environ['PYSPARK_DRIVER_PYTHON'] = python_path

# Add Spark and Python paths
sys.path.append(os.path.join(spark_home, 'python'))
sys.path.append(os.path.join(spark_home, 'python', 'lib', 'py4j-0.10.9.7-src.zip'))

# spark = get_spark_session()

from pyspark.sql import SparkSession
import os
# from config import *

# def get_spark_session():
spark = SparkSession.builder \
    .appName("GCS Connector Test") \
    .config("spark.pyspark.python", python_path) \
    .config("spark.pyspark.driver.python", python_path) \
    .config("spark.driver.extraLibraryPath", os.path.dirname(python_path)) \
    .config("spark.driver.extraClassPath", gcs_connector_jar) \
    .config("spark.executor.extraClassPath", gcs_connector_jar) \
    .config("spark.local.dir", "D:/Empty/spark-temp") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", KEY_FILE) \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.python.worker.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .master("local[*]") \
    .getOrCreate()
            
    # return spark



In [3]:
path = "gs://crypto-historical-data-2/ver2/ADA/2021/03/data.parquet"
df = spark.read.parquet(path)
df.show()

+----+----------+------+------+------+------+--------------+----+-----+-----------------+
|BASE|      DATE|  OPEN|  HIGH|   LOW| CLOSE|        VOLUME|YEAR|MONTH|__index_level_0__|
+----+----------+------+------+------+------+--------------+----+-----+-----------------+
| ADA|2021-03-31|1.2135|1.2138|1.1526|1.1934| 4.601137893E7|2021|    3|             1335|
| ADA|2021-03-30|1.2032|1.2365|1.1888|1.2135| 4.517666711E7|2021|    3|             1336|
| ADA|2021-03-29|1.1903|1.2294|1.1777|1.2032|  4.34716022E7|2021|    3|             1337|
| ADA|2021-03-28|1.1768|1.2188|1.1666|1.1903| 3.909916673E7|2021|    3|             1338|
| ADA|2021-03-27|1.2144|1.2415|1.1575|1.1768| 4.792371762E7|2021|    3|             1339|
| ADA|2021-03-26|1.0963|1.2869|1.0944|1.2144| 7.374575707E7|2021|    3|             1340|
| ADA|2021-03-25|1.0674|1.1547|1.0442|1.0963| 7.470805297E7|2021|    3|             1341|
| ADA|2021-03-24|1.1178|1.1686|  1.04|1.0674| 6.046568588E7|2021|    3|             1342|
| ADA|2021

In [5]:
df.schema

StructType([StructField('BASE', StringType(), True), StructField('DATE', StringType(), True), StructField('OPEN', DoubleType(), True), StructField('HIGH', DoubleType(), True), StructField('LOW', DoubleType(), True), StructField('CLOSE', DoubleType(), True), StructField('VOLUME', DoubleType(), True), StructField('YEAR', IntegerType(), True), StructField('MONTH', IntegerType(), True), StructField('__index_level_0__', LongType(), True)])

In [6]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DoubleType, IntegerType, LongType
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

gcs_jar_path = os.path.abspath("config/gcs-connector-hadoop3-latest.jar")
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
# spark = SparkSession.builder \
#     .appName("hehee") \
#     .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")\
#     .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")\
#     .config("spark.hadoop.fs.gs.auth.service.account.enable", "true")\
#     .config("spark.hadoop.fs.gs.auth.service.account.json.keyfile", "config/key/btcanalysishust-495a3a227f22.json") \
#     .config("spark.jars", gcs_jar_path) \
#     .config("spark.jars.packages", 
#             "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,io.delta:delta-core_2.12:2.2.0") \
#     .config("spark.hadoop.fs.gs.project.id", "btcanalysishust")\
#     .getOrCreate()


column_names = ["BTC", "ETH", "USDT", "USDC", "XRP", "ADA", "DOGE", "MATIC", "SOL"]

schema = StructType([
    StructField("BASE", StringType(), True),  
    StructField("DATE", StringType(), True),
    StructField("OPEN", DoubleType(), True),
    StructField("HIGH", DoubleType(), True),
    StructField("LOW", DoubleType(), True),
    StructField("CLOSE", DoubleType(), True),
    StructField("VOLUME", DoubleType(), True),
    StructField("YEAR", IntegerType(), True),
    StructField("MONTH", IntegerType(), True),
    StructField("__index_level_0__", LongType(), True)
])

import datetime

def read_historical_data(coin):
    current_year = datetime.datetime.now().year  
    years = [str(year) for year in range(2021, current_year + 1)]  

    paths = [f"gs://crypto-historical-data-2/ver2/{coin}/{year}/*" for year in years]
    
    return (
        spark.read.schema(schema).parquet(*paths)  
        .select(F.col("DATE").cast("timestamp"), "CLOSE")
    )

def process_coin(coin):
    historical_data_df = read_historical_data(coin)
    window_spec = Window.orderBy("DATE").rowsBetween(Window.unboundedPreceding, 0)
    
    historical_data_df = historical_data_df.withColumn(f"SMA_5", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-4, 0)))
    historical_data_df = historical_data_df.withColumn(f"SMA_10", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-9, 0)))
    historical_data_df = historical_data_df.withColumn(f"SMA_20", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-19, 0)))
    historical_data_df = historical_data_df.withColumn(f"SMA_50", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-49, 0)))
    historical_data_df = historical_data_df.withColumn(f"SMA_100", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-99, 0)))
    historical_data_df = historical_data_df.withColumn(f"SMA_200", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-199, 0)))

    historical_data_df = historical_data_df.select("DATE", "CLOSE", "SMA_5", "SMA_10", "SMA_20", "SMA_50", "SMA_100", "SMA_200").orderBy("DATE", ascending=False)
    return historical_data_df
    
    # tmp_dir = f"gs://indicator-crypto/sma_results/batch/{coin}"

    # historical_data_df.write \
    #     .format("csv") \
    #     .option("header", "true") \
    #     .option("path", tmp_dir) \
    #     .mode("append") \
    #     .save()

# with ThreadPoolExecutor() as executor:
#     futures = [executor.submit(process_coin, coin) for coin in column_names]
#     for future in as_completed(futures):
#         print(future.result())

coin = column_names[0]
process_coin(coin).show()

+-------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|               DATE|   CLOSE|            SMA_5|           SMA_10|           SMA_20|           SMA_50|          SMA_100|          SMA_200|
+-------------------+--------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|2024-12-13 00:00:00| 99897.8|         99035.98|         99196.71|97502.35500000001|86719.17599999996|74509.28739999999|       68734.2753|
|2024-12-12 00:00:00|100050.1|         99290.42|98798.68000000001|97458.66650000002|86053.24219999996|       74084.9978|68577.15489999998|
|2024-12-11 00:00:00|101177.0|         99260.38|          98379.0|97378.11600000001|85399.84179999998|       73675.8951|68423.35274999999|
|2024-12-10 00:00:00| 96655.1|98999.55999999998|97907.84199999999|       97038.9235|84723.22479999997|       73237.1151|68260.20674999998|
|2024-12-09 00:00:00| 97399

In [12]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DoubleType, IntegerType, LongType
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
import datetime

# Danh sách các cột coin
column_names = ["BTC", "ETH", "USDT", "USDC", "XRP", "ADA", "DOGE", "MATIC", "SOL"]

# Schema của dữ liệu đầu vào
schema = StructType([
    StructField("BASE", StringType(), True),  
    StructField("DATE", StringType(), True),
    StructField("OPEN", DoubleType(), True),
    StructField("HIGH", DoubleType(), True),
    StructField("LOW", DoubleType(), True),
    StructField("CLOSE", DoubleType(), True),
    StructField("VOLUME", DoubleType(), True),
    StructField("YEAR", IntegerType(), True),
    StructField("MONTH", IntegerType(), True),
    StructField("__index_level_0__", LongType(), True)
])

def read_historical_data(coin):
    current_year = datetime.datetime.now().year  
    years = [str(year) for year in range(2021, current_year + 1)]  

    paths = [f"gs://crypto-historical-data-2/ver2/{coin}/{year}/*" for year in years]
    
    return (
        spark.read.schema(schema).parquet(*paths)  
        .select(F.col("DATE").cast("timestamp"), "CLOSE")
    )

from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# Hàm Python tính EMA
def calculate_ema_udf(period):
    alpha = 2 / (period + 1)
    
    def calculate_ema(values):
        ema_values = []
        for i, close in enumerate(values):
            if i == 0:
                ema_values.append(close)  # Giá trị EMA đầu tiên là CLOSE đầu tiên
            else:
                ema = alpha * close + (1 - alpha) * ema_values[-1]
                ema_values.append(ema)
        return round(ema_values[-1], 2)  # Trả về giá trị EMA cuối cùng trong danh sách

    return udf(lambda values: calculate_ema(values), DoubleType())

# Thêm cột EMA bằng UDF
def calculate_ema(df, column, period):
    window_spec = Window.orderBy("DATE").rowsBetween(Window.unboundedPreceding, 0)
    df = df.withColumn(f"{column}_LIST", F.collect_list(column).over(window_spec))
    ema_udf = calculate_ema_udf(period)
    return df.withColumn(f"EMA_{period}", ema_udf(F.col(f"{column}_LIST"))).drop(f"{column}_LIST")


def process_coin(coin):
    historical_data_df = read_historical_data(coin)

    # Tính toán EMA cho các khoảng thời gian
    ema_periods = [5, 10, 20, 50, 100, 200]
    for period in ema_periods:
        historical_data_df = calculate_ema(historical_data_df, "CLOSE", period)

    historical_data_df = historical_data_df.select("DATE", "CLOSE", *[f"EMA_{period}" for period in ema_periods]).orderBy("DATE", ascending=False)


    tmp_dir = f"gs://indicator-crypto/ema_results/batch/{coin}"

    historical_data_df.write \
        .format("csv") \
        .option("header", "true") \
        .option("path", tmp_dir) \
        .mode("append") \
        .save()

    return historical_data_df

with ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_coin, coin) for coin in column_names]
    for future in as_completed(futures):
        print(future.result())


DataFrame[DATE: timestamp, CLOSE: double, EMA_5: double, EMA_10: double, EMA_20: double, EMA_50: double, EMA_100: double, EMA_200: double]
DataFrame[DATE: timestamp, CLOSE: double, EMA_5: double, EMA_10: double, EMA_20: double, EMA_50: double, EMA_100: double, EMA_200: double]
DataFrame[DATE: timestamp, CLOSE: double, EMA_5: double, EMA_10: double, EMA_20: double, EMA_50: double, EMA_100: double, EMA_200: double]
DataFrame[DATE: timestamp, CLOSE: double, EMA_5: double, EMA_10: double, EMA_20: double, EMA_50: double, EMA_100: double, EMA_200: double]
DataFrame[DATE: timestamp, CLOSE: double, EMA_5: double, EMA_10: double, EMA_20: double, EMA_50: double, EMA_100: double, EMA_200: double]
DataFrame[DATE: timestamp, CLOSE: double, EMA_5: double, EMA_10: double, EMA_20: double, EMA_50: double, EMA_100: double, EMA_200: double]
DataFrame[DATE: timestamp, CLOSE: double, EMA_5: double, EMA_10: double, EMA_20: double, EMA_50: double, EMA_100: double, EMA_200: double]
DataFrame[DATE: timestamp, 

In [ ]:
from pyspark import SparkContext 
sc = SparkContext.getOrCreate() 

In [3]:
import os
import sys

# Explicitly set Python and library paths
python_path = sys.executable
spark_home = r"D:\Empty\spark-3.5.3-bin-hadoop3-scala2.13\spark-3.5.3-bin-hadoop3-scala2.13"  # Adjust to your Spark installation path
hadoop_home = r"D:\Empty\hadoop\hadoop"
java_home = r"D:\Empty\Java"

KEY_FILE = r"D:\Empty\btcanalysishust-d7c3a4830bef.json"
bucket_name = 'ba_bucket3'
gcs_connector_jar = os.path.join(hadoop_home, 'share', 'hadoop', 'tools', 'lib', 'gcs-connector-hadoop3-latest.jar')

# Set environment variables
os.environ['SPARK_HOME'] = spark_home
os.environ['HADOOP_HOME'] = hadoop_home
os.environ['JAVA_HOME'] = java_home
os.environ['PYSPARK_PYTHON'] = python_path
os.environ['PYSPARK_DRIVER_PYTHON'] = python_path

# Add Spark and Python paths
sys.path.append(os.path.join(spark_home, 'python'))
sys.path.append(os.path.join(spark_home, 'python', 'lib', 'py4j-0.10.9.7-src.zip'))

# spark = get_spark_session()

from pyspark.sql import SparkSession
import os
# from config import *

# def get_spark_session():
# spark = SparkSession.builder \
#     .appName("GCS Connector Test") \
#     .config("spark.pyspark.python", python_path) \
#     .config("spark.pyspark.driver.python", python_path) \
#     .config("spark.driver.extraLibraryPath", os.path.dirname(python_path)) \
#     .config("spark.driver.extraClassPath", gcs_connector_jar) \
#     .config("spark.executor.extraClassPath", gcs_connector_jar) \
#     .config("spark.hadoop.fs.gs.auth.service.account.enable", "true")\
#     .config("spark.local.dir", "D:/Empty/spark-temp") \
#     .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
#     .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
#     .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", KEY_FILE) \
#     .config("spark.jars.packages", 
#             "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,io.delta:delta-core_2.12:2.2.0") \
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
#     .config("spark.driver.memory", "4g") \
#     .config("spark.executor.memory", "4g") \
#     .config("spark.python.worker.memory", "2g") \
#     .config("spark.sql.shuffle.partitions", "200") \
#     .master("local[*]") \
#     .getOrCreate()
            
    # return spark

# spark = SparkSession.builder \
#     .appName("GCS Connector Test") \
#     .config("spark.pyspark.python", python_path) \
#     .config("spark.pyspark.driver.python", python_path) \
#     .config("spark.driver.extraLibraryPath", os.path.dirname(python_path)) \
#     .config("spark.driver.extraClassPath", gcs_connector_jar) \
#     .config("spark.executor.extraClassPath", gcs_connector_jar) \
#     .config("spark.hadoop.fs.gs.auth.service.account.enable", "true")\
#     .config("spark.local.dir", "D:/Empty/spark-temp") \
#     .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
#     .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
#     .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", KEY_FILE) \
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
#     .config("spark.jars.packages", 
#             "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,"
#             "io.delta:delta-core_2.12:2.2.0,"
#             "io.delta:delta-spark_2.12:2.2.0") \
#     .config("spark.driver.memory", "4g") \
#     .config("spark.executor.memory", "4g") \
#     .config("spark.python.worker.memory", "2g") \
#     .config("spark.sql.shuffle.partitions", "200") \
#     .master("local[*]") \
#     .getOrCreate()

In [11]:
spark.stop()
# spark = SparkSession.builder \
#     .master("local") \
#     .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") \
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
#     .getOrCreate()

spark = SparkSession.builder \
    .appName("hehee") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")\
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")\
    .config("spark.hadoop.fs.gs.auth.service.account.enable", "true")\
    .config("spark.hadoop.fs.gs.auth.service.account.json.keyfile", "config/key/btcanalysishust-495a3a227f22.json") \
    .config("spark.jars", gcs_connector_jar) \
    .config("spark.jars.packages", 
        "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.3,io.delta:delta-core_2.13:2.4.0") \
    .config("spark.hadoop.fs.gs.project.id", "btcanalysishust")\
    .getOrCreate()

# from delta.tables import *

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MinimalTest") \
    .getOrCreate()

df = spark.createDataFrame([(1, "Alice"), (2, "Bob")], ["id", "name"])
df.show()

spark.stop()


+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
+---+-----+



In [1]:
# pip install kafka-python

In [13]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DoubleType, IntegerType, LongType
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
from pyspark.sql.types import StructType, StructField, StringType, FloatType
import pyspark.sql.functions as F
# from delta import *
# from delta.tables import *


# gcs_jar_path = os.path.abspath("config/gcs-connector-hadoop3-latest.jar")
# from pyspark.sql import SparkSession

# # Khởi tạo SparkSession
# spark = SparkSession.builder \
#     .appName("hehee") \
#     .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")\
#     .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")\
#     .config("spark.hadoop.fs.gs.auth.service.account.enable", "true")\
#     .config("spark.hadoop.fs.gs.auth.service.account.json.keyfile", "config/key/btcanalysishust-495a3a227f22.json") \
#     .config("spark.jars", gcs_jar_path) \
#     .config("spark.jars.packages", 
#             "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,io.delta:delta-core_2.12:2.2.0") \
#     .config("spark.hadoop.fs.gs.project.id", "btcanalysishust")\
#     .getOrCreate()

# Schema của dữ liệu Kafka
schema = StructType([
    StructField("timestamp", StringType()),
    StructField("prices", StructType([
        StructField("bitcoin", FloatType()),
        StructField("ethereum", FloatType()),
        StructField("tether", FloatType()),
        StructField("usd-coin", FloatType()),
        StructField("ripple", FloatType()),
        StructField("cardano", FloatType()),
        StructField("dogecoin", FloatType()),
        StructField("matic-network", FloatType()),
        StructField("solana", FloatType()),
        StructField("litecoin", FloatType()),
        StructField("polkadot", FloatType()),
        StructField("shiba-inu", FloatType()),
        StructField("tron", FloatType()),
        StructField("cosmos", FloatType()),
        StructField("chainlink", FloatType()),
        StructField("stellar", FloatType()),
        StructField("near", FloatType()),
    ]))
])

# Đọc dữ liệu từ Kafka
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "35.206.252.44:9092") \
    .option("subscribe", "crypto-pricessss") \
    .option("startingOffsets", "latest") \
    .option("maxOffsetsPerTrigger", 1000) \
    .load()

parsed_df = kafka_df.selectExpr("CAST(value AS STRING)") \
    .select(F.from_json(F.col("value"), schema).alias("data"))

crypto_parsed_df = parsed_df.select(
    F.to_timestamp(F.col("data.timestamp"), "yyyy-MM-dd'T'HH:mm:ss.SSSSSSXXX").alias("DATE"),
    F.col("data.prices.bitcoin").alias("BTC"),
    F.col("data.prices.ethereum").alias("ETH"),
    F.col("data.prices.tether").alias("USDT"),
    F.col("data.prices.usd-coin").alias("USDC"),
    F.col("data.prices.ripple").alias("XRP"),
    F.col("data.prices.cardano").alias("ADA"),
    F.col("data.prices.dogecoin").alias("DOGE"),
    F.col("data.prices.matic-network").alias("MATIC"),
    F.col("data.prices.solana").alias("SOL")
)
crypto_parsed_df.show()

# column_names = [col for col in crypto_parsed_df.columns if col != 'DATE']

# historical_schema = StructType([
#     StructField("BASE", StringType(), True),  
#     StructField("DATE", StringType(), True),
#     StructField("OPEN", DoubleType(), True),
#     StructField("HIGH", DoubleType(), True),
#     StructField("LOW", DoubleType(), True),
#     StructField("CLOSE", DoubleType(), True),
#     StructField("VOLUME", DoubleType(), True),
#     StructField("YEAR", IntegerType(), True),
#     StructField("MONTH", IntegerType(), True),
#     StructField("__index_level_0__", LongType(), True)
# ])


# def read_historical_data(coin):
#     return (
#         spark.read.schema(historical_schema).format("parquet")
#         .load(f"gs://crypto-historical-data-2/ver2/{coin}/2024/*")
#         .select(F.col("DATE").cast("timestamp"), "CLOSE")
#     )

# def process_coin(coin, micro_batch_latest_df):
#     historical_data_df = read_historical_data(coin)
#     micro_batch = micro_batch_latest_df.select("DATE", coin).withColumnRenamed(coin, "CLOSE")
#     combined_df = micro_batch.unionByName(historical_data_df)

#     window_spec = Window.orderBy("DATE").rowsBetween(Window.unboundedPreceding, 0)
#     combined_df = combined_df.withColumn(f"SMA_5", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-4, 0)))
#     combined_df = combined_df.withColumn(f"SMA_10", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-9, 0)))
#     combined_df = combined_df.withColumn(f"SMA_20", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-19, 0)))
#     combined_df = combined_df.withColumn(f"SMA_50", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-49, 0)))
#     combined_df = combined_df.withColumn(f"SMA_100", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-99, 0)))
#     combined_df = combined_df.withColumn(f"SMA_200", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-199, 0)))

#     combined_df = combined_df.orderBy("DATE", ascending=False)
    
#     current_date = F.current_date() 
#     combined_df = combined_df.filter(F.col("DATE") >= current_date)

#     tmp_dir = f"gs://indicator-crypto/sma_results/tmp/{coin}"

#     #combined_df.write.format("console").option("truncate", False).save()
#     # combined_df.write \
#     #     .format("csv") \
#     #     .option("header", "true") \
#     #     .option("path", tmp_dir) \
#     #     .mode("append") \
#     #     .save()
#     return combined_df
    
# def process_batch(micro_batch_df, batch_id):
#     micro_batch_latest_df = (
#         micro_batch_df
#         .withColumn("row_num", F.row_number().over(Window.orderBy(F.col("DATE").desc())))
#         .filter(F.col("row_num") == 1)
#         .drop("row_num")
#     )
    
#     with ThreadPoolExecutor() as executor:
#         futures = [executor.submit(process_coin, coin, micro_batch_latest_df) for coin in column_names]
#         for future in as_completed(futures):
#             print(future.result()) 

# # Thực thi stream
# query = crypto_parsed_df.writeStream \
#     .foreachBatch(process_batch) \
#     .start()

# query.awaitTermination()



Py4JJavaError: An error occurred while calling o273.load.
: java.util.ServiceConfigurationError: org.apache.spark.sql.sources.DataSourceRegister: org.apache.spark.sql.delta.sources.DeltaDataSource Unable to get public no-arg constructor
	at java.base/java.util.ServiceLoader.fail(ServiceLoader.java:582)
	at java.base/java.util.ServiceLoader.getConstructor(ServiceLoader.java:673)
	at java.base/java.util.ServiceLoader$LazyClassPathLookupIterator.hasNextService(ServiceLoader.java:1233)
	at java.base/java.util.ServiceLoader$LazyClassPathLookupIterator.hasNext(ServiceLoader.java:1265)
	at java.base/java.util.ServiceLoader$2.hasNext(ServiceLoader.java:1300)
	at java.base/java.util.ServiceLoader$3.hasNext(ServiceLoader.java:1385)
	at scala.collection.convert.JavaCollectionWrappers$JIteratorWrapper.hasNext(JavaCollectionWrappers.scala:37)
	at scala.collection.StrictOptimizedIterableOps.filterImpl(StrictOptimizedIterableOps.scala:225)
	at scala.collection.StrictOptimizedIterableOps.filt

In [1]:
import pyspark
print(pyspark.__version__)

3.5.3


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, FloatType

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DoubleType, IntegerType, LongType
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
from pyspark.sql.types import StructType, StructField, StringType, FloatType
import pyspark.sql.functions as F
# from delta import *

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("KafkaToSpark") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.3") \
    .getOrCreate()

# Định nghĩa schema
schema = StructType([
    StructField("timestamp", StringType()),
    StructField("prices", StructType([
        StructField("bitcoin", FloatType()),
        StructField("ethereum", FloatType()),
        StructField("tether", FloatType()),
        StructField("usd-coin", FloatType()),
        StructField("ripple", FloatType()),
        StructField("cardano", FloatType()),
        StructField("dogecoin", FloatType()),
        StructField("matic-network", FloatType()),
        StructField("solana", FloatType()),
        StructField("litecoin", FloatType()),
        StructField("polkadot", FloatType()),
        StructField("shiba-inu", FloatType()),
        StructField("tron", FloatType()),
        StructField("cosmos", FloatType()),
        StructField("chainlink", FloatType()),
        StructField("stellar", FloatType()),
        StructField("near", FloatType()),
    ]))
])

# Đọc dữ liệu từ Kafka
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "35.206.252.44:9092") \
    .option("subscribe", "crypto-pricessss") \
    .option("startingOffsets", "latest") \
    .option("maxOffsetsPerTrigger", 1000) \
    .load()

# Parse dữ liệu JSON
parsed_df = kafka_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data"))

# Select relevant columns for cryptocurrencies
crypto_parsed_df = parsed_df.select(
    F.to_timestamp(F.col("data.timestamp"), "yyyy-MM-dd'T'HH:mm:ss.SSSSSSXXX").alias("DATE"),
    F.col("data.prices.bitcoin").alias("BTC"),
    F.col("data.prices.ethereum").alias("ETH"),
    F.col("data.prices.tether").alias("USDT"),
    F.col("data.prices.usd-coin").alias("USDC"),
    F.col("data.prices.ripple").alias("XRP"),
    F.col("data.prices.cardano").alias("ADA"),
    F.col("data.prices.dogecoin").alias("DOGE"),
    F.col("data.prices.matic-network").alias("MATIC"),
    F.col("data.prices.solana").alias("SOL")
)


# Xuất ra console
# query = crypto_parsed_df.writeStream \
#     .format("console") \
#     .start()
# query = crypto_parsed_df.writeStream \
#     .format("console") \
#     .outputMode("append") \
#     .start()

# # query.awaitTermination()  # Đảm bảo stream chạy liên tục
# query = crypto_parsed_df.writeStream \
#     .format("console") \
#     .outputMode("append") \
#     .trigger(processingTime="10 seconds") \  # Chạy mỗi 10 giây
#     .start()

# query.awaitTermination()

def process_batch(df, epoch_id):
    print(f"Processing batch {epoch_id}")
    df.show()  # Hoặc thực hiện các hành động khác

query = crypto_parsed_df.writeStream \
    .foreachBatch(process_batch) \
    .start()

query.awaitTermination()

# query.awaitTermination()


Processing batch 0
+----+---+---+----+----+---+---+----+-----+---+
|DATE|BTC|ETH|USDT|USDC|XRP|ADA|DOGE|MATIC|SOL|
+----+---+---+----+----+---+---+----+-----+---+
+----+---+---+----+----+---+---+----+-----+---+

Processing batch 1
+--------------------+--------+------+--------+--------+----+-----+--------+-------+------+
|                DATE|     BTC|   ETH|    USDT|    USDC| XRP|  ADA|    DOGE|  MATIC|   SOL|
+--------------------+--------+------+--------+--------+----+-----+--------+-------+------+
|2024-12-14 19:38:...|101350.0|3876.9|0.999873|0.999313|2.42|1.087|0.402838|0.60537|222.74|
+--------------------+--------+------+--------+--------+----+-----+--------+-------+------+

Processing batch 2
+--------------------+--------+------+--------+--------+----+-----+--------+-------+------+
|                DATE|     BTC|   ETH|    USDT|    USDC| XRP|  ADA|    DOGE|  MATIC|   SOL|
+--------------------+--------+------+--------+--------+----+-----+--------+-------+------+
|2024-12-14 1

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, FloatType

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DoubleType, IntegerType, LongType
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
from pyspark.sql.types import StructType, StructField, StringType, FloatType
import pyspark.sql.functions as F
# from delta import *

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("KafkaToSpark") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.3") \
    .getOrCreate()

# Định nghĩa schema
schema = StructType([
    StructField("timestamp", StringType()),
    StructField("prices", StructType([
        StructField("bitcoin", FloatType()),
        StructField("ethereum", FloatType()),
        StructField("tether", FloatType()),
        StructField("usd-coin", FloatType()),
        StructField("ripple", FloatType()),
        StructField("cardano", FloatType()),
        StructField("dogecoin", FloatType()),
        StructField("matic-network", FloatType()),
        StructField("solana", FloatType()),
        StructField("litecoin", FloatType()),
        StructField("polkadot", FloatType()),
        StructField("shiba-inu", FloatType()),
        StructField("tron", FloatType()),
        StructField("cosmos", FloatType()),
        StructField("chainlink", FloatType()),
        StructField("stellar", FloatType()),
        StructField("near", FloatType()),
    ]))
])

# Đọc dữ liệu từ Kafka
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "35.206.252.44:9092") \
    .option("subscribe", "crypto-pricessss") \
    .option("startingOffsets", "latest") \
    .option("maxOffsetsPerTrigger", 1000) \
    .load()

# Parse dữ liệu JSON
parsed_df = kafka_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data"))

# Select relevant columns for cryptocurrencies
crypto_parsed_df = parsed_df.select(
    F.to_timestamp(F.col("data.timestamp"), "yyyy-MM-dd'T'HH:mm:ss.SSSSSSXXX").alias("DATE"),
    F.col("data.prices.bitcoin").alias("BTC"),
    F.col("data.prices.ethereum").alias("ETH"),
    F.col("data.prices.tether").alias("USDT"),
    F.col("data.prices.usd-coin").alias("USDC"),
    F.col("data.prices.ripple").alias("XRP"),
    F.col("data.prices.cardano").alias("ADA"),
    F.col("data.prices.dogecoin").alias("DOGE"),
    F.col("data.prices.matic-network").alias("MATIC"),
    F.col("data.prices.solana").alias("SOL")
)


# Xuất ra console
# query = crypto_parsed_df.writeStream \
#     .format("console") \
#     .start()
# query = crypto_parsed_df.writeStream \
#     .format("console") \
#     .outputMode("append") \
#     .start()

# # query.awaitTermination()  # Đảm bảo stream chạy liên tục
# query = crypto_parsed_df.writeStream \
#     .format("console") \
#     .outputMode("append") \
#     .trigger(processingTime="10 seconds") \  # Chạy mỗi 10 giây
#     .start()

# query.awaitTermination()

def process_batch(df, epoch_id):
    print(f"Processing batch {epoch_id}")
    df.show()  # Hoặc thực hiện các hành động khác

query = crypto_parsed_df.writeStream \
    .foreachBatch(process_batch) \
    .start()

query.awaitTermination()

# query.awaitTermination()


Processing batch 0
+----+---+---+----+----+---+---+----+-----+---+
|DATE|BTC|ETH|USDT|USDC|XRP|ADA|DOGE|MATIC|SOL|
+----+---+---+----+----+---+---+----+-----+---+
+----+---+---+----+----+---+---+----+-----+---+



In [ ]:
query = crypto_parsed_df.writeStream \
    .foreachBatch(process_batch) \
    .trigger(processingTime="10 seconds") \  # Cập nhật trigger để chạy mỗi 10 giây
    .start()

query.awaitTermination()


In [1]:
print("")

In [ ]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DoubleType, IntegerType, LongType
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

# gcs_jar_path = os.path.abspath("config/gcs-connector-hadoop3-latest.jar")
from pyspark.sql import SparkSession
hadoop_home = r"D:\Empty\hadoop\hadoop"
KEY_FILE = r"D:\Empty\btcanalysishust-d7c3a4830bef.json"
gcs_connector_jar = os.path.join(hadoop_home, 'share', 'hadoop', 'tools', 'lib', 'gcs-connector-hadoop3-latest.jar')

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("hehee") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")\
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")\
    .config("spark.hadoop.fs.gs.auth.service.account.enable", "true")\
    .config("spark.hadoop.fs.gs.auth.service.account.json.keyfile", KEY_FILE) \
    .config("spark.jars", gcs_connector_jar) \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.3") \
    .config("spark.hadoop.fs.gs.project.id", "btcanalysishust")\
    .getOrCreate()

# Schema của dữ liệu Kafka
schema = StructType([
    StructField("timestamp", StringType()),
    StructField("prices", StructType([
        StructField("bitcoin", FloatType()),
        StructField("ethereum", FloatType()),
        StructField("tether", FloatType()),
        StructField("usd-coin", FloatType()),
        StructField("ripple", FloatType()),
        StructField("cardano", FloatType()),
        StructField("dogecoin", FloatType()),
        StructField("matic-network", FloatType()),
        StructField("solana", FloatType()),
        StructField("litecoin", FloatType()),
        StructField("polkadot", FloatType()),
        StructField("shiba-inu", FloatType()),
        StructField("tron", FloatType()),
        StructField("cosmos", FloatType()),
        StructField("chainlink", FloatType()),
        StructField("stellar", FloatType()),
        StructField("near", FloatType()),
    ]))
])

# Đọc dữ liệu từ Kafka
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "35.206.252.44:9092") \
    .option("subscribe", "crypto-pricessss") \
    .option("startingOffsets", "latest") \
    .option("maxOffsetsPerTrigger", 1000) \
    .load()
# kafka_df.show()

# parsed_df = kafka_df.selectExpr("CAST(value AS STRING)") \
#     .select(F.from_json(F.col("value"), schema).alias("data"))

# crypto_parsed_df = parsed_df.select(
#     F.to_timestamp(F.col("data.timestamp"), "yyyy-MM-dd'T'HH:mm:ss.SSSSSSXXX").alias("DATE"),
#     F.col("data.prices.bitcoin").alias("BTC"),
#     F.col("data.prices.ethereum").alias("ETH"),
#     F.col("data.prices.tether").alias("USDT"),
#     F.col("data.prices.usd-coin").alias("USDC"),
#     F.col("data.prices.ripple").alias("XRP"),
#     F.col("data.prices.cardano").alias("ADA"),
#     F.col("data.prices.dogecoin").alias("DOGE"),
#     F.col("data.prices.matic-network").alias("MATIC"),
#     F.col("data.prices.solana").alias("SOL")
# )

# column_names = [col for col in crypto_parsed_df.columns if col != 'DATE']

# historical_schema = StructType([
#     StructField("BASE", StringType(), True),  
#     StructField("DATE", StringType(), True),
#     StructField("OPEN", DoubleType(), True),
#     StructField("HIGH", DoubleType(), True),
#     StructField("LOW", DoubleType(), True),
#     StructField("CLOSE", DoubleType(), True),
#     StructField("VOLUME", DoubleType(), True),
#     StructField("YEAR", IntegerType(), True),
#     StructField("MONTH", IntegerType(), True),
#     StructField("__index_level_0__", LongType(), True)
# ])


# def read_historical_data(coin):
#     return (
#         spark.read.schema(historical_schema).format("parquet")
#         .load(f"gs://crypto-historical-data-2/ver2/{coin}/2024/*")
#         .select(F.col("DATE").cast("timestamp"), "CLOSE")
#     )

# def process_coin(coin, micro_batch_latest_df):
#     historical_data_df = read_historical_data(coin)
#     micro_batch = micro_batch_latest_df.select("DATE", coin).withColumnRenamed(coin, "CLOSE")
#     combined_df = micro_batch.unionByName(historical_data_df)

#     window_spec = Window.orderBy("DATE").rowsBetween(Window.unboundedPreceding, 0)
#     combined_df = combined_df.withColumn(f"SMA_5", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-4, 0)))
#     combined_df = combined_df.withColumn(f"SMA_10", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-9, 0)))
#     combined_df = combined_df.withColumn(f"SMA_20", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-19, 0)))
#     combined_df = combined_df.withColumn(f"SMA_50", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-49, 0)))
#     combined_df = combined_df.withColumn(f"SMA_100", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-99, 0)))
#     combined_df = combined_df.withColumn(f"SMA_200", F.avg(F.col("CLOSE")).over(window_spec.rowsBetween(-199, 0)))

#     combined_df = combined_df.orderBy("DATE", ascending=False)
    
#     current_date = F.current_date() 
#     combined_df = combined_df.filter(F.col("DATE") >= current_date)

#     # tmp_dir = f"gs://indicator-crypto/sma_results/tmp/{coin}"

#     # #combined_df.write.format("console").option("truncate", False).save()
#     # combined_df.write \
#     #     .format("csv") \
#     #     .option("header", "true") \
#     #     .option("path", tmp_dir) \
#     #     .mode("append") \
#     #     .save()
#     return combined_df
    
# def process_batch(micro_batch_df, batch_id):
#     micro_batch_latest_df = (
#         micro_batch_df
#         .withColumn("row_num", F.row_number().over(Window.orderBy(F.col("DATE").desc())))
#         .filter(F.col("row_num") == 1)
#         .drop("row_num")
#     )
#     # coin = column_names[0]
#     # process_coin(coin, micro_batch_latest_df).show()
#     with ThreadPoolExecutor() as executor:
#         futures = [executor.submit(process_coin, coin, micro_batch_latest_df) for coin in column_names]
#         for future in as_completed(futures):
#             print(future.result()) 
#             result_df = future.result()  # Processed DataFrame for each coin
#             result_df.show(truncate=False)  # Display the DataFrame without truncating content
            
            
# # process_batch(crypto_parsed_df).show()
# # Thực thi stream
def test(micro_batch_df, batch_id):
    # In dữ liệu của micro_batch_df
    micro_batch_df.show(truncate=False)

# Thực thi stream và chỉ in dữ liệu
query = kafka_df.writeStream \
    .foreachBatch(test) \
    .start()

query.awaitTermination()

# query = kafka_df.writeStream \
#     .foreachBatch(test) \
#     .start()

# query.awaitTermination()

+---+-----+-----+---------+------+---------+-------------+
|key|value|topic|partition|offset|timestamp|timestampType|
+---+-----+-----+---------+------+---------+-------------+
+---+-----+-----+---------+------+---------+-------------+



In [ ]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DoubleType, IntegerType, LongType
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
import datetime

hadoop_home = r"D:\Empty\hadoop\hadoop"
KEY_FILE = r"D:\Empty\btcanalysishust-d7c3a4830bef.json"
gcs_connector_jar = os.path.join(hadoop_home, 'share', 'hadoop', 'tools', 'lib', 'gcs-connector-hadoop3-latest.jar')

# def get_spark_session():
spark = SparkSession.builder \
    .appName("GCS Connector Test") \
    .config("spark.driver.extraClassPath", gcs_connector_jar) \
    .config("spark.executor.extraClassPath", gcs_connector_jar) \
    .config("spark.local.dir", "D:/Empty/spark-temp") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", KEY_FILE) \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.3") \
    .master("local[*]") \
    .getOrCreate()
            
    
# Schema của dữ liệu Kafka
schema = StructType([
    StructField("timestamp", StringType()),
    StructField("prices", StructType([
        StructField("bitcoin", FloatType()),
        StructField("ethereum", FloatType()),
        StructField("tether", FloatType()),
        StructField("usd-coin", FloatType()),
        StructField("ripple", FloatType()),
        StructField("cardano", FloatType()),
        StructField("dogecoin", FloatType()),
        StructField("matic-network", FloatType()),
        StructField("solana", FloatType()),
        StructField("litecoin", FloatType()),
        StructField("polkadot", FloatType()),
        StructField("shiba-inu", FloatType()),
        StructField("tron", FloatType()),
        StructField("cosmos", FloatType()),
        StructField("chainlink", FloatType()),
        StructField("stellar", FloatType()),
        StructField("near", FloatType()),
    ]))
])

# Đọc dữ liệu từ Kafka
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "35.206.252.44:9092") \
    .option("subscribe", "crypto-pricessss") \
    .option("startingOffsets", "latest") \
    .option("maxOffsetsPerTrigger", 1000) \
    .load()

parsed_df = kafka_df.selectExpr("CAST(value AS STRING)") \
    .select(F.from_json(F.col("value"), schema).alias("data"))

crypto_parsed_df = parsed_df.select(
    F.to_timestamp(F.col("data.timestamp"), "yyyy-MM-dd'T'HH:mm:ss.SSSSSSXXX").alias("DATE"),
    F.col("data.prices.bitcoin").alias("BTC"),
    F.col("data.prices.ethereum").alias("ETH"),
    F.col("data.prices.tether").alias("USDT"),
    F.col("data.prices.usd-coin").alias("USDC"),
    F.col("data.prices.ripple").alias("XRP"),
    F.col("data.prices.cardano").alias("ADA"),
    F.col("data.prices.dogecoin").alias("DOGE"),
    F.col("data.prices.matic-network").alias("MATIC"),
    F.col("data.prices.solana").alias("SOL")
)

column_names = [col for col in crypto_parsed_df.columns if col != 'DATE']

historical_schema = StructType([
    StructField("BASE", StringType(), True),  
    StructField("DATE", StringType(), True),
    StructField("OPEN", DoubleType(), True),
    StructField("HIGH", DoubleType(), True),
    StructField("LOW", DoubleType(), True),
    StructField("CLOSE", DoubleType(), True),
    StructField("VOLUME", DoubleType(), True),
    StructField("YEAR", IntegerType(), True),
    StructField("MONTH", IntegerType(), True),
    StructField("__index_level_0__", LongType(), True)
])


def read_historical_data(coin):
    return (
        spark.read.schema(historical_schema).format("parquet")
        .load(f"gs://crypto-historical-data-2/ver2/{coin}/2024/*")
        .select(F.col("DATE").cast("timestamp"), "CLOSE")
    )
    
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType
from pyspark.sql.window import Window
from pyspark.sql import SparkSession

# Hàm tính EMA
def calculate_ema(df, column, period):
    alpha = 2 / (period + 1)
    window_spec = Window.orderBy("DATE").rowsBetween(Window.unboundedPreceding, 0)
    
    # Tính giá trị EMA
    ema_column = F.avg(F.col(column).cast(DoubleType())).over(window_spec)
    ema_values = df.withColumn(f"EMA_{period}", alpha * F.col(column) + (1 - alpha) * ema_column)
    print(ema_values)
    return ema_values

def process_coin(coin, micro_batch_latest_df):
    historical_data_df = read_historical_data(coin)
    micro_batch = micro_batch_latest_df.select("DATE", coin).withColumnRenamed(coin, "CLOSE")
    
    # Kết hợp dữ liệu hiện tại với dữ liệu lịch sử
    combined_df = micro_batch.unionByName(historical_data_df)
    
    # Tính toán EMA
    ema_periods = [5, 10, 20, 50, 100, 200]
    for period in ema_periods:
        combined_df = calculate_ema(combined_df, "CLOSE", period)
    
    # Sắp xếp và lọc dữ liệu theo ngày
    combined_df = combined_df.orderBy("DATE", ascending=False)
    current_date = F.current_date() 
    combined_df = combined_df.filter(F.col("DATE") >= current_date)
    
    # Lưu kết quả vào GCS
    tmp_dir = f"gs://indicator-crypto/ema_results/tmp/{coin}"
    combined_df.write \
        .format("csv") \
        .option("header", "true") \
        .option("path", tmp_dir) \
        .mode("append") \
        .save()


def process_batch(micro_batch_df, batch_id):
    micro_batch_latest_df = (
        micro_batch_df
        .withColumn("row_num", F.row_number().over(Window.orderBy(F.col("DATE").desc())))
        .filter(F.col("row_num") == 1)
        .drop("row_num")
    )
    
    # Sử dụng ThreadPoolExecutor để xử lý nhiều đồng tiền song song
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_coin, coin, micro_batch_latest_df) for coin in column_names]
        for future in as_completed(futures):
            print(future.result())

# Chạy job streaming
query = crypto_parsed_df.writeStream \
    .foreachBatch(process_batch) \
    .start()

query.awaitTermination()
